<a href="https://colab.research.google.com/github/DimpleB0501/eva8/blob/main/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function # this has to be at the top. A future statement is a directive to the compiler that a particular module should be compiled using syntax or semantics that will be available in a specified future release of Python where the feature becomes standard. The future statement is intended to ease migration to future versions of Python that introduce incompatible changes to the language.
import torch # import pytorch 
import torch.nn as nn # import all the functions that are used in neural networks models. https://pytorch.org/tutorials/beginner/nn_tutorial.html
import torch.nn.functional as F #https://pytorch.org/docs/stable/nn.functional.html
import torch.optim as optim # Implemented optimization functions. https://pytorch.org/docs/stable/optim.html
from torchvision import datasets, transforms # The torchvision package consists of popular datasets, model architectures, and common image transformations for computer vision.

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input - 28 | Output - 28 | RF - 3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1) # input - 28 | Ouput - 28 | RF -5
        self.pool1 = nn.MaxPool2d(2, 2) # input - 28 | Ouput - 14 | RF - 10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1) # input - 14 | Ouput - 14 | RF - 12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1) # input - 14 | Ouput - 14 | RF - 14
        self.pool2 = nn.MaxPool2d(2, 2) #input - 14 | Ouput - 7 | RF - 28
        self.conv5 = nn.Conv2d(256, 512, 3) #  input - 7 | Ouput - 5 | RF - 30
        self.conv6 = nn.Conv2d(512, 1024, 3) # input - 5 | Ouput - 3 | RF - 32
        self.conv7 = nn.Conv2d(1024, 10, 3) # input - 3 | Ouput - 1 | RF - 34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = F.relu(self.conv7(x))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available() # check if the system is on cpu or gpu
device = torch.device("cuda" if use_cuda else "cpu") # select the device as gpu if available 
model = Net().to(device) # put the convolutional  network to the device 
# https://towardsdatascience.com/understanding-and-calculating-the-number-of-parameters-in-convolution-neural-networks-cnns-fc88790d530d
# 1 conv layer ((m * n * d)+1)* k) i.e. ((shape of width of the filter * shape of height of the filter * number of filters in the previous layer+1)*number of filters)
# so conv1 ((3*3*1) + 1) * 32 = 320
# conv2 ((3*3*32)+1) * 64 = 18496
summary(model, input_size=(1, 28, 28)) 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 

<ipython-input-2-7645d8171ebb>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [4]:


torch.manual_seed(1)
batch_size = 128 # datasize is 128 while processing it in batches

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader( # load the training and apply transforms like normalization and converting it to a tensor
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))  # use mean of and std deivation of training set
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [5]:
!pip install torchscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from torchscan import summary
summary(model,(1, 28, 28), receptive_field=True, max_depth=1)

_________________________________________________________________________
Layer      Type         Output Shape         Param #      Receptive field
net        Net          (-1, 10)             0            1              
├─conv1    Conv2d       (-1, 32, 28, 28)     320          3              
├─conv2    Conv2d       (-1, 64, 28, 28)     18,496       5              
├─pool1    MaxPool2d    (-1, 64, 14, 14)     0            6              
├─conv3    Conv2d       (-1, 128, 14, 14)    73,856       10             
├─conv4    Conv2d       (-1, 256, 14, 14)    295,168      14             
├─pool2    MaxPool2d    (-1, 256, 7, 7)      0            16             
├─conv5    Conv2d       (-1, 512, 5, 5)      1,180,160    24             
├─conv6    Conv2d       (-1, 1024, 3, 3)     4,719,616    32             
├─conv7    Conv2d       (-1, 10, 1, 1)       92,170       40             
Trainable params: 6,379,786
Non-trainable params: 0
Total params: 6,379,786
------------------------------------

<ipython-input-2-7645d8171ebb>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [7]:
from tqdm import tqdm # training models with progress bars
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader) # progress depends on data and batch size
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device) # upload data and labels to gpu (or cpu)
        optimizer.zero_grad() # https://pytorch.org/docs/stable/generated/torch.optim.Optimizer.zero_grad.html, optimizer.zero_grad() clears x.grad for every parameter x in the optimizer. It’s important to call this before loss.backward(), otherwise you’ll accumulate the gradients from multiple passes.
        output = model(data) # process the data through the models
        loss = F.nll_loss(output, target) # calculate the loss
        loss.backward() #  computes dloss/dx for every parameter x which has requires_grad=True. These are accumulated into x.grad for every parameter x. In pseudo-code:x.grad += dloss/dx
        optimizer.step() # perform parameter update, optimizer.step updates the value of x using the gradient x.grad. For example, the SGD optimizer performs:x += -lr * x.grad

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
model = Net().to(device) # put the model to device
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # using sgd optimizer https://pytorch.org/docs/stable/generated/torch.optim.SGD.html

for epoch in range(1, 2): # number of times the train needs to be performed
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-2-7645d8171ebb>:21: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.02540111355483532 batch_id=468: 100%|██████████| 469/469 [00:20<00:00, 22.49it/s]



Test set: Average loss: 0.0662, Accuracy: 9792/10000 (98%)

